In [12]:
# Import libraries
import pandas as pd
import sys

In [21]:
####### 2019 #########
####### APRIL ########

# List of file paths
file_paths = ['data/2019/april/calendar.csv', 'data/2019/april/listings.csv', 'data/2019/april/listings0.csv',
              'data/2019/april/neighbourhoods.csv', 'data/2019/april/reviews.csv', 'data/2019/april/reviews0.csv']

#file_paths = ['data/2019/april/listings.csv', 'data/2019/april/listings0.csv']

# Initialize an empty list to store 'id' columns from each filef
id_columns = []

# Iterate through each file path
for file_path in file_paths:
    # Read the 'id' column from each file
    df = pd.read_csv(file_path, usecols=['id'])
    # Append the 'id' column to the list
    id_columns.append(df)

# Concatenate 'id' columns from all files
all_ids = pd.concat(id_columns, ignore_index=True)

# Get unique IDs
unique_ids = all_ids['id'].unique()

# Create a new DataFrame with unique IDs
unique_ids_df = pd.DataFrame({'id': unique_ids})

# Specify the path for the new CSV file
new_file_path = 'data_train/train_2019.csv'

# Write the DataFrame with unique IDs to a new CSV file
unique_ids_df.to_csv(new_file_path, index=False)

print("New CSV file created successfully with unique IDs.")

ValueError: Usecols do not match columns, columns expected but not found: ['id']